# chess

In [2]:
from pettingzoo.classic import chess_v6

In [2]:
# install dependencies
!git clone https://github.com/rlarena/notebook_blueprint.git > /dev/null 2>&1
!pip install -r notebook_blueprint/requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
env = chess_v6.env()
env.reset(seed=42)
cum_reward = {agent:0 for agent in env.agents}

# interact with env
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    cum_reward[agent] += reward
    if termination or truncation:
        action = None
    else:
        mask = observation["action_mask"]
        # this is where you would insert your policy
        action = env.action_space(agent).sample(mask)

    env.step(action)
env.close()
print(cum_reward)

{'player_0': 0, 'player_1': 0}


## Train agent

In [4]:
model_path = "my_model.zip"

In [5]:
import glob
import os
import time

from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker

import pettingzoo.utils
from pettingzoo.classic import chess_v6


class SB3ActionMaskWrapper(pettingzoo.utils.BaseWrapper):
    """Wrapper to allow PettingZoo environments to be used with SB3 illegal action masking."""

    def reset(self, seed=None, options=None):
        """Gymnasium-like reset function which assigns obs/action spaces to be the same for each agent.

        This is required as SB3 is designed for single-agent RL and doesn't expect obs/action spaces to be functions
        """
        super().reset(seed, options)

        # Strip the action mask out from the observation space
        self.observation_space = super().observation_space(self.possible_agents[0])[
            "observation"
        ]
        self.action_space = super().action_space(self.possible_agents[0])

        # Return initial observation, info (PettingZoo AEC envs do not by default)
        return self.observe(self.agent_selection), {}

    def step(self, action):
        """Gymnasium-like step function, returning observation, reward, termination, truncation, info."""
        super().step(action)
        return super().last()

    def observe(self, agent):
        """Return only raw observation, removing action mask."""
        return super().observe(agent)["observation"]

    def action_mask(self):
        """Separate function used in order to access the action mask."""
        return super().observe(self.agent_selection)["action_mask"]


def mask_fn(env):
    return env.action_mask()


def train_action_mask(env_fn, steps=10_000, seed=0, **env_kwargs):
    """Train a single model to play as each agent in a zero-sum game environment using invalid action masking."""
    env = env_fn.env(**env_kwargs)

    print(f"Starting training on {str(env.metadata['name'])}.")

    # Custom wrapper to convert PettingZoo envs to work with SB3 action masking
    env = SB3ActionMaskWrapper(env)

    env.reset(seed=seed)  # Must call reset() in order to re-define the spaces

    env = ActionMasker(env, mask_fn)  # Wrap to enable masking (SB3 function)
    # MaskablePPO behaves the same as SB3's PPO unless the env is wrapped
    # with ActionMasker. If the wrapper is detected, the masks are automatically
    # retrieved and used when learning. Note that MaskablePPO does not accept
    # a new action_mask_fn kwarg, as it did in an earlier draft.
    model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)
    model.set_random_seed(seed)
    model.learn(total_timesteps=steps)

    model.save(model_path)

    print("Model has been saved.")

    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.\n")

    env.close()


def eval_action_mask(env_fn, num_games=100, render_mode=None, **env_kwargs):
    # Evaluate a trained agent vs a random agent
    env = env_fn.env(render_mode=render_mode, **env_kwargs)

    print(
        f"Starting evaluation vs a random agent. Trained agent will play as {env.possible_agents[1]}."
    )

    try:
        latest_policy = max(
            glob.glob(f"{env.metadata['name']}*.zip"), key=os.path.getctime
        )
    except ValueError:
        print("Policy not found.")
        exit(0)

    model = MaskablePPO.load(latest_policy)

    scores = {agent: 0 for agent in env.possible_agents}
    total_rewards = {agent: 0 for agent in env.possible_agents}
    round_rewards = []

    for i in range(num_games):
        env.reset(seed=i)
        env.action_space(env.possible_agents[0]).seed(i)

        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()

            # Separate observation and action mask
            observation, action_mask = obs.values()

            if termination or truncation:
                # If there is a winner, keep track, otherwise don't change the scores (tie)
                if (
                    env.rewards[env.possible_agents[0]]
                    != env.rewards[env.possible_agents[1]]
                ):
                    winner = max(env.rewards, key=env.rewards.get)
                    scores[winner] += env.rewards[
                        winner
                    ]  # only tracks the largest reward (winner of game)
                # Also track negative and positive rewards (penalizes illegal moves)
                for a in env.possible_agents:
                    total_rewards[a] += env.rewards[a]
                # List of rewards by round, for reference
                round_rewards.append(env.rewards)
                break
            else:
                if agent == env.possible_agents[0]:
                    act = env.action_space(agent).sample(action_mask)
                else:
                    # Note: PettingZoo expects integer actions # TODO: change chess to cast actions to type int?
                    act = int(
                        model.predict(
                            observation, action_masks=action_mask, deterministic=True
                        )[0]
                    )
            env.step(act)
    env.close()

    # Avoid dividing by zero
    if sum(scores.values()) == 0:
        winrate = 0
    else:
        winrate = scores[env.possible_agents[1]] / sum(scores.values())
    print("Rewards by round: ", round_rewards)
    print("Total rewards (incl. negative rewards): ", total_rewards)
    print("Winrate: ", winrate)
    print("Final scores: ", scores)
    return round_rewards, total_rewards, winrate, scores

In [ ]:
env_fn = chess_v6
env_kwargs = {}
train_action_mask(env_fn, steps=20_480, seed=0, **env_kwargs)

Starting training on chess_v6.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/raphael/.cache/pypoetry/virtualenvs/notebook-blueprint-S0X6msc3-py3.10/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 270      |
|    ep_rew_mean     | -0.286   |
| time/              |          |
|    fps             | 291      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 311         |
|    ep_rew_mean          | -0.154      |
| time/                   |             |
|    fps                  | 203         |
|    iterations           | 2           |
|    time_elapsed         | 20          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.034519576 |
|    clip_fraction        | 0.474       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.98       |
|    explained_variance   | -0.622      |
|    learning_rate        | 0.

In [22]:
# Evaluate 100 games against a random agent
res = eval_action_mask(env_fn, num_games=100, render_mode=None, **env_kwargs)

Starting evaluation vs a random agent. Trained agent will play as player_1.
Rewards by round:  [{'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': -1, 'player_1': 1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 1, 'player_1': -1}, {'player_0': 

## Publish agent

In [19]:
class Agent:
    def __init__(self, env):
        self.env = env
        from stable_baselines3 import DQN
        self.model =  DQN.load(model_path)

    def choose_action(self, observation, action_mask=None):
        action, _states = self.model.predict(observation, deterministic=True)
        return action

In [30]:
my_stringify_agent = f"""
class Agent:
    def __init__(self, env):
        self.env = env
        from stable_baselines3 import DQN
        self.model =  DQN.load("{model_path}")

    def choose_action(self, observation, action_mask=None):
        action, _states = self.model.predict(observation, deterministic=True)
        return action
"""

In [31]:
# go to https://rlarena.com/my-profile to get or generate your key pair
key_id=""
key_pass=""
#
agent_attach_name= "my_super_agent"

In [32]:
import requests
import io

def attach_agent(key_id, key_pass, agent_attach_name, model_path):
    # Endpoint URL
    url = 'https://rlarena.com/api/direct_attache_agents_notebook/competition/4'

    # Your credentials and agent details
    data = {
        'key_id': key_id,
        'key_pass': key_pass,
        'agent_attach_name': agent_attach_name,
    }

    # Files to upload: agent.py and the model
    files = {
        'model': (model_path, open(model_path, 'rb')),
        'agent_code': ('agent.py', io.StringIO(my_stringify_agent)),
    }

    # Make the POST request
    try:
        response = requests.post(url, data=data, files=files)
        # Ensure files are closed properly
        files['model'][1].close()
        files['agent_code'][1].close()
        return response.json()  # Return the JSON response
    except Exception as e:
        print(f"Failed to attach agent due to: {str(e)}")
        return None


response = attach_agent(key_id, key_pass, agent_attach_name, model_path)
print(response)

{'attache_agent_id': 6, 'message': 'Uploaded successfully: my_model.zip, agent.py'}
